In [3]:
import numpy
from urllib.request import urlopen
import scipy.optimize
import random
from collections import defaultdict
import string
import gzip
import pandas as pd
import sklearn
from sklearn import linear_model

The unique bigrams are 521502
    The top bigrams are
    [(7927, ('of', 'the')),
     (5850, ('this', 'book')),
     (5627, ('in', 'the')),
     (3189, ('and', 'the')),
     (3183, ('is', 'a'))]
    

In [4]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

data = []
for d in readGz("train_Category.json.gz"):
    data.append(d)

df = pd.DataFrame(data)

data = df.iloc[:10000]
data_review = data['review_text']

In [7]:
bigram = defaultdict(int)
punctuation = set(string.punctuation)
for d in data_review:
    r = ''.join([c for c in d.lower() if not c in punctuation])
    words = r.split()
    for j in range(0, len(words)-1):
        bigram[words[j], words[j+1]] +=1
        

In [8]:
counts = [(bigram[w], w) for w in bigram]
counts.sort()
counts.reverse()

In [9]:
counts[:5]

[(7927, ('of', 'the')),
 (5850, ('this', 'book')),
 (5627, ('in', 'the')),
 (3189, ('and', 'the')),
 (3183, ('is', 'a'))]

In [10]:
len(bigram.keys())

521502

In [11]:
words = [x[1] for x in counts[:1000]]

### Sentiment analysis

wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

def feature(datum):
    feat = [0]*len(words)
    r = ''.join([c for c in datum.lower() if not c in punctuation])
    wo = r.split()
    for j in range(0, len(wo)-1): 
        w = wo[j], wo[j+1]
        if w in words:
            feat[wordId[w]] += 1
    feat.append(1) #offset
    return feat

X = [feature(d) for d in data_review]
y = [d for d in data['rating']]

In [12]:
# Regularized regression
clf = linear_model.Ridge(1.0, fit_intercept=False) # MSE + 1.0 l2
clf.fit(X, y)
theta = clf.coef_
predictions = clf.predict(X)

In [13]:
import sklearn.metrics

In [14]:
mse = sklearn.metrics.mean_squared_error(y, predictions)
print(mse)

1.0178804824879226


In [40]:
uniwordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in data_review:
    r = ''.join([c for c in d.lower() if not c in punctuation])
    for wuni in r.split():
        uniwordCount[wuni] += 1
unicounts = [(uniwordCount[wc], wc) for wc in uniwordCount]

bigramc = defaultdict(int)
punctuation = set(string.punctuation)
for d in data_review:
    r = ''.join([c for c in d.lower() if not c in punctuation])
    words = r.split()
    for j in range(0, len(words)-1):
        bigramc[words[j] + ' '+ words[j+1]] +=1
        
counts = [(bigramc[w], w) for w in bigramc]    
#print(counts[0])
uni_bi = unicounts + counts
uni_bi.sort()
uni_bi.reverse()  
#print(uni_bi[:10])

[(73431, 'the'), (44301, 'and'), (39577, 'a'), (36821, 'to'), (36581, 'i'), (32552, 'of'), (21889, 'is'), (21468, 'in'), (20110, 'it'), (19353, 'this')]


In [50]:
uni_biwords = [x[1] for x in uni_bi[:1000]]

### Sentiment analysis

wordIdub = dict(zip(uni_biwords, range(len(uni_biwords))))
wordSetub = set(uni_biwords)

def feature(datum):
    feat = [0]*len(uni_biwords)
    r = ''.join([c for c in datum.lower() if not c in punctuation])
    wo = r.split()
    for j, i in zip(range(0, len(wo)-1), wo): 
        w = wo[j] + ' '+ wo[j+1]
        if w in uni_biwords:
            feat[wordIdub[w]] += 1
        if i in uni_biwords:
            feat[wordIdub[i]] += 1
    feat.append(1) #offset
    return feat

X = [feature(d) for d in data_review]
y = [d for d in data['rating']]

In [51]:
clf = linear_model.Ridge(1.0, fit_intercept=False) # MSE + 1.0 l2
clf.fit(X, y)
theta = clf.coef_
predictions = clf.predict(X)
mse = sklearn.metrics.mean_squared_error(y, predictions)
print(mse)

0.9704424465275538


In [52]:
data = []
for d in readGz("train_Category.json.gz"):
    data.append(d)

df = pd.DataFrame(data)

data = df.iloc[:10000]
data_review = data['review_text']

In [46]:
import math
def tf(t, d):
  #  word = []
   ## for i in d.split():
    #    words = []
     #   word .append(i)
    #print(word)
    #wordCount = len(d.split())
    count = 0
    d = ''.join([c for c in d.lower() if not c in punctuation])
    for i in d.split():
        if t == i:
            #print(t)
            count += 1
    return float(count)

def idf(t, file):
    df = 0
    #count = 0
    count = len(file)
    for i in file:
        i = ''.join([c for c in i.lower() if not c in punctuation])
        #print(t)
        if t in i:
            #count+= 1
            #print(count)
            df +=1 
    return math.log((float(count)/float((df))), 10)
    
for i in ['stories', 'magician', 'psychic', 'writing', 'wonder']:
    print(i , tf(i, data_review[0])*idf(i, data_review))

#print(data_review[0])

stories 1.1018235165023234
magician 2.4685210829577446
psychic 5.075204004202088
writing 0.9901243662878397
wonder 1.09151498112135


In [134]:
data = []
for d in readGz("train_Category.json.gz"):
    data.append(d)

df = pd.DataFrame(data)

data = df.iloc[:10000]
data_review = data['review_text']



In [135]:
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in data_review:
    r = ''.join([c for c in d.lower() if not c in punctuation])
    for w in r.split():
        wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

words = [x[1] for x in counts[:1000]]


In [185]:
#words

In [186]:
#words[569] in data_review

In [201]:
idfwords = []
for i in words:
    #print(i)
    j = idf(i, data_review)
    #print(j)
    idfwords.append(j)
    #print(idfwords)

In [203]:
#idfwords

In [205]:
### Sentiment analysis

wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

def feature(datum):
    feat = [0]*len(words)
    for i, j in zip(words, idfwords):
        #print(tf(i, datum))
        feat.append(tf(i, datum)* j)
    feat.append(1) #offset
    return feat

X = [feature(d) for d in data_review]
y = [d for d in data['rating']]

In [ ]:
#print(idfwords)

In [153]:
#X[0][:20]

In [ ]:
#X[0][:20]

In [152]:
#X[0][:30]

In [ ]:
#X[0][:30]

In [180]:
#for i in X[0]:
    #print(i > 0)

In [208]:
clf = linear_model.Ridge(1.0, fit_intercept=False) # MSE + 1.0 l2
clf.fit(X, y)
theta = clf.coef_
predictions = clf.predict(X)
mse = sklearn.metrics.mean_squared_error(y, predictions)
print(mse)

0.9672305813829284


In [209]:
def cosinsim (vector, v):
    magnitude = math.sqrt(sum(n**2 for n in vector))
    mag = math.sqrt(sum(n**2 for n in v))
    dot = sum(n*m for n, m in zip(vector, v))
    m = magnitude*mag
    answer = dot/m
    return answer

In [210]:
max_cosin = 0
index = 0
#for i, j in zip(X, range(0, len(X))):
for i in range(1, len(X)):
    answers = cosinsim(X[0], X[i])
    #$print(answers)
    if answers > max_cosin:
        max_cosin = answers
        index = i

In [211]:
index

6921

In [214]:
max_cosin

0.33740223573513983

In [149]:
#data_review[9985]

In [150]:
#data_review[0]

In [212]:
data_review[6921]

'This review is going to be very personal. I will probably not talk about the writing, the plot or the characters the way that I normally would. Dear Martin is about 17-year-old Justyce McAllistera a highly successful black student who attends a primarily all-white private school. One day, he is helping a drunk friend and arrested. In response, he starts writing letters to Dr, Martin Luther King Jr. as a way to cope with his newly sharpened awareness of the world around him \n The world needed this book right now. This is a book that every school needs. I am going to be buying copies for friends and family. I am a teacher. I can see how this book would be an excellent way to help students start having these difficult conversations. Because until we teach young people how to recognize and talk about racism, sexism, homophobia, and xenophobia and whatever else needs to get talked about nothing will change. They will unconsciously internalize the system\'s biases. They won\'t see the thin

In [151]:
#data_review[6922]

In [5]:
data = []
for d in readGz("train_Category.json.gz"):
    data.append(d)

df = pd.DataFrame(data)

data = df.iloc[:30000]
data_reviewq = data['review_text']

In [6]:
#import random
#random.shuffle(data_reviewq)

In [7]:
train = data_reviewq[:10000]
validation = data_reviewq[10000:20000]
test = data_reviewq[20000:]

In [229]:
len(train)
len(validation)
len(test)

10000

In [155]:
import re
#bigram without pun
bigram = defaultdict(int)
punctuation = set(string.punctuation)
for d in data_review:
    r = ''.join([c for c in d.lower() if not c in punctuation])
    words = r.split()
    for j in range(0, len(words)-1):
        bigram[words[j], words[j+1]] +=1
counts = [(bigram[w], w) for w in bigram]
counts.sort()
counts.reverse()

#unigram without pun
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in data:
    r = ''.join([c for c in d.lower() if not c in punctuation])
    for w in r.split():
        wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

words = [x[1] for x in counts[:1000]]

#bigram with pun
bigramwp = defaultdict(int)
punctuation = list(string.punctuation)
for d in data_review:
    r = ''.join([c for c in d.lower()])
    words =  [t.strip() for t in re.findall(r'\b.*?\S.*?(?:\b|$)', r)]
    for j in range(0, len(words)-1):
        bigramwp[words[j], words[j+1]] +=1
countswp = [(bigramwp[w], w) for w in bigramwp]
countswp.sort()
countswp.reverse()

#unigram with pun
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in data:
    r = ''.join([c for c in d.lower() if not c in punctuation])
    for w in [t.strip() for t in re.findall(r'\b.*?\S.*?(?:\b|$)', r)]:
        wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

KeyboardInterrupt: 

In [246]:
#import re
#re.findall(r"[\w']+|['!','"',"#",'$','%','&',"'",'(',')','*','+',',','-','.','/',':',';','<','=','>','?','@','[','\\',']','^','_','`','{','|','}','~']", "Hello, I'm a string!")

In [2]:
>>> import re
>>> sentence = 'Hello, world!!'
>>> tokens = [t.strip() for t in re.findall(r'\b.*?\S.*?(?:\b|$)', sentence)]
['Hello', ',', 'world', '!']

['Hello', ',', 'world', '!']

In [3]:
import re
sentence = 'Hello, world!!'
tokens = [t.strip() for t in re.findall(r'\b.*?\S.*?(?:\b|$)', sentence)]

In [4]:
print(tokens)

['Hello', ',', 'world', '!!']


In [5]:
import re
sentence = 'Hello, world!!'
tokens = [t.strip() for t in re.findall(r"[\w']+|[.,!?;]", sentence)]
tokens

['Hello', ',', 'world', '!', '!']

In [ ]:
# split it into sets
# words with punc bi
# wors no punc bi
# words with punc uni
# words no punc uni
\# tfidf
# bag of words

# Model Bigrams with bag of words

In [23]:
bigram = defaultdict(int)
punctuation = set(string.punctuation)
for d in data_reviewq:
    r = ''.join([c for c in d.lower()])
    words = [t.strip() for t in re.findall(r"[\w']+|[.,!?;]", r)]
    for j in range(0, len(words)-1):
        bigram[words[j], words[j+1]] +=1
        
counts = [(bigram[w], w) for w in bigram]
counts.sort()
counts.reverse()

words = [x[1] for x in counts[:1000]]



In [13]:
#@words

In [24]:
### Sentiment analysis

wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

def feature(datum):
    feat = [0]*len(words)
    r = ''.join([c for c in datum.lower()])
    wo = [t.strip() for t in re.findall(r"[\w']+|[.,!?;]", r)]
    for j in range(0, len(wo)-1): 
        w = wo[j], wo[j+1]
        if w in words:
            feat[wordId[w]] += 1
    feat.append(1) #offset
    return feat

X = [feature(d) for d in train]
X_vali = [feature(d) for d in validation]
X_test = [feature(d) for d in test]
y = [d for d in data['rating']]


In [25]:
y_train = [d for d in data['rating'][:10000]]

In [118]:
mse_bibagwp = []
# Regularized regression
for c in [0.01, 0.1, 1, 10, 100]:
    clf = linear_model.Ridge(c, fit_intercept=False) # MSE + 1.0 l2
    clf.fit(X, y_train)
    theta = clf.coef_
    predictions = clf.predict(X_vali)
    mse = sklearn.metrics.mean_squared_error(y_train, predictions)
    mse_bibagwp.append([mse,c])
    print(mse, c)

1.3453519100143996 0.01
1.345351911439946 0.1
1.3453520539686856 1
1.3453662809917355 10
1.3467635116165084 100


In [27]:
mse_bibagwp

[[1.7633384790508368, 0.01],
 [1.7630522819206949, 0.1],
 [1.7602603926428755, 1],
 [1.7367797543001453, 10],
 [1.6344206961136214, 100]]

# Model bigram less bag of words

In [28]:
bigram = defaultdict(int)
punctuation = set(string.punctuation)
for d in data_reviewq:
    r = ''.join([c for c in d.lower() if not c in punctuation])
    words = r.split()
    for j in range(0, len(words)-1):
        bigram[words[j], words[j+1]] +=1
counts = [(bigram[w], w) for w in bigram]
counts.sort()
counts.reverse()
words = [x[1] for x in counts[:1000]]

In [30]:
### Sentiment analysis

wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

def feature(datum):
    feat = [0]*len(words)
    r = ''.join([c for c in datum.lower() if not c in punctuation])
    wo = r.split()
    for j in range(0, len(wo)-1): 
        w = wo[j], wo[j+1]
        if w in words:
            feat[wordId[w]] += 1
    feat.append(1) #offset
    return feat

X = [feature(d) for d in train]
X_vali = [feature(d) for d in validation]
X_test = [feature(d) for d in test]

In [33]:
y_train = [d for d in data['rating'][:10000]]

In [119]:
mse_bibag = []
# Regularized regression
for c in [0.01, 0.1, 1, 10, 100]:
    clf = linear_model.Ridge(c, fit_intercept=False) # MSE + 1.0 l2
    clf.fit(X, y_train)
    theta = clf.coef_
    predictions = clf.predict(X_vali)
    mse = sklearn.metrics.mean_squared_error(y_train, predictions)
    mse_bibag.append([mse,c, 'bibag'])
    print(mse, c)

1.3453519100143996 0.01
1.345351911439946 0.1
1.3453520539686856 1
1.3453662809917355 10
1.3467635116165084 100


In [35]:
mse_bibag

[[1.7535714973555219, 0.01, 'bibag'],
 [1.7532701639541546, 0.1, 'bibag'],
 [1.7503232446051902, 1, 'bibag'],
 [1.7253947206708733, 10, 'bibag'],
 [1.6179981892725532, 100, 'bibag']]

# Model unigram with bag of word

In [36]:
uniwordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in data_reviewq:
    r = ''.join([c for c in d.lower() if not c in punctuation])
    for wuni in [t.strip() for t in re.findall(r"[\w']+|[.,!?;]", r)]:
        uniwordCount[wuni] += 1
unicounts = [(uniwordCount[wc], wc) for wc in uniwordCount]
unicounts.sort()
unicounts.reverse()
words = [x[1] for x in unicounts[:1000]]

In [37]:
### Sentiment analysis

wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

def feature(datum):
    feat = [0]*len(words)
    r = ''.join([c for c in datum.lower() if not c in punctuation])
    for w in [t.strip() for t in re.findall(r"[\w']+|[.,!?;]", r)]:
        if w in words:
            feat[wordId[w]] += 1
    feat.append(1) #offset
    return feat

X = [feature(d) for d in train]
X_vali = [feature(d) for d in validation]
X_test = [feature(d) for d in test]
y_train = [d for d in data['rating'][:10000]]


In [120]:
mse_ubagwp = []
# Regularized regression
for c in [0.01, 0.1, 1, 10, 100]:
    clf = linear_model.Ridge(c, fit_intercept=False) # MSE + 1.0 l2
    clf.fit(X, y_train)
    theta = clf.coef_
    predictions = clf.predict(X_vali)
    mse = sklearn.metrics.mean_squared_error(y_train, predictions)
    mse_ubagwp.append([mse,c, 'ubagwp'])
    print(mse, c)

1.3453519100143996 0.01
1.345351911439946 0.1
1.3453520539686856 1
1.3453662809917355 10
1.3467635116165084 100


# Model unigram less bag of words

In [8]:
uniwordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in data_reviewq:
    r = ''.join([c for c in d.lower() if not c in punctuation])
    for wuni in r.split():
        uniwordCount[wuni] += 1
unicounts = [(uniwordCount[wc], wc) for wc in uniwordCount]
unicounts.sort()
unicounts.reverse()
words = [x[1] for x in unicounts[:1000]]

In [9]:
### Sentiment analysis

wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

def feature(datum):
    feat = [0]*len(words)
    r = ''.join([c for c in d.lower() if not c in punctuation])
    for wuni in r.split():
        if wuni in words:
            feat[wordId[wuni]] += 1
    feat.append(1) #offset
    return feat

X = [feature(d) for d in train]
X_vali = [feature(d) for d in validation]
X_test = [feature(d) for d in test]
y_train = [d for d in data['rating'][:10000]]


In [121]:
mse_unibag = []
# Regularized regression
for c in [0.01, 0.1, 1, 10, 100]:
    clf = linear_model.Ridge(c, fit_intercept=False) # MSE + 1.0 l2
    clf.fit(X, y_train)
    theta = clf.coef_
    predictions = clf.predict(X_vali)
    mse = sklearn.metrics.mean_squared_error(y_train, predictions)
    mse_unibag.append([mse,c, 'unibag'])
    print(mse, c)

1.3453519100143996 0.01
1.345351911439946 0.1
1.3453520539686856 1
1.3453662809917355 10
1.3467635116165084 100


In [137]:
#lst = []
#for j in X:
 #   count = 0
  #  for i in j:
   #     if int(i) != 0:
    #        count = count +1
    #lst.append(count)
#print(count)
#print(lst)

# Model bigrams with tfidf

In [76]:
def idfp(t, file):
    df = 0
    #count = 0
    count = len(file)
    for i in file:
        i = ''.join([c for c in i.lower()])
        #print(t)
        if t in i:
            #count+= 1
            #print(count)
            df +=1 
    return math.log((float(count)/float((df))), 10)

In [103]:
def idfp(t, file):
    df = 0
    #count = 0
    count = len(file)
    for i in file:
        i = ''.join([c for c in i.lower()])
        #print(t)
        if t in i:
            #count+= 1
            #print(count)
            df +=1 
        if df == 0:
            df = -1
    return math.log((float(count)/float((df))), 10)

In [105]:
def idfp(t, file):
    df = 0
    #count = 0
    count = len(file)
    for i in file:
        i = ''.join([c for c in i.lower()])
        #print(t)
        if t in i:
            #count+= 1
            #print(count)
            df +=1 
        if df == 0:
            df = 1
    return math.log((float(count)/float((df))), 10)

In [106]:
def idfp(t, file):
    df = 0
    #count = 0
    count = len(file)
    for i in file:
        i = ''.join([c for c in i.lower()])
        #print(t)
        if t in i:
            #count+= 1
            #print(count)
            df +=1 
    if df == 0:
        return -1
    else:
        return math.log((float(count)/float((df))), 10)

In [41]:
#X[0]

In [67]:
bigram = defaultdict(int)
punctuation = set(string.punctuation)
for d in data_reviewq:
    r = ''.join([c for c in d.lower()])
    words = [t.strip() for t in re.findall(r"[\w']+|[.,!?;]", r)]
    for j in range(0, len(words)-1):
        bigram[words[j] + ' '+ words[j+1]] +=1
        
counts = [(bigram[w], w) for w in bigram]
counts.sort()
counts.reverse()

words = [x[1] for x in counts[:1000]]


In [42]:
#X[1]

In [82]:
#train[0]

In [78]:
#words[0] in 'happy. i'

In [79]:
#words[0]

In [80]:
#a = 'Happy!'
#'!' in a

In [81]:
#words

In [138]:
#words[43]

In [139]:
#words[44]

In [140]:
#words[44] in train

In [97]:
#for i in train:
 #   print(words[43] in train)

In [99]:
for i in train:
    i = ''.join([c for c in i.lower()])
    if words[42] in i == True:
        print(words[43] in train)
    

In [102]:
for i in train:
    i = ''.join([c for c in i.lower()])
    if words[40] in i == True:
        print(words[42] in train)

In [ ]:
for i in train:
    if words[42] in train == True:
        print(words[43] in train)

In [108]:
idfwords = []
for i in words:
    #print(i)
    j = idfp(i, train)
    #print(j)
    idfwords.append(j)
    #print(idfwords)

In [109]:
wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

def feature(datum):
    feat = [0]*len(words)
    for i, j in zip(words, idfwords):
        #print(tf(i, datum))
        feat.append(tf(i, datum)* j)
    feat.append(1) #offset
    return feat

X = [feature(d) for d in train]
y_train = [d for d in data['rating'][:10000]]

In [111]:
idfwordsvali = []
for i in words:
    #print(i)
    j = idfp(i, validation)
    #print(j)
    idfwordsvali.append(j)

In [112]:
def featurevali(datum):
    feat = [0]*len(words)
    for i, j in zip(words, idfwordsvali):
        #print(tf(i, datum))
        feat.append(tf(i, datum)* j)
    feat.append(1) #offset
    return feat
X_vali = [featurevali(d) for d in validation]

In [135]:
idfwordstest = []
for i in words:
    #print(i)
    j = idfp(i, test)
    #print(j)
    idfwordstest.append(j)

In [136]:
def featuretest(datum):
    feat = [0]*len(words)
    for i, j in zip(words, idfwordstest):
        #print(tf(i, datum))
        feat.append(tf(i, datum)* j)
    feat.append(1) #offset
    return feat
X_test = [featuretest(d) for d in test]

In [ ]:
### Sentiment analysis

wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

def feature(datum):
    feat = [0]*len(words)
    for i, j in zip(words, idfwords):
        #print(tf(i, datum))
        feat.append(tf(i, datum)* j)
    feat.append(1) #offset
    return feat

X = [feature(d) for d in train]
X_vali = [feature(d) for d in validation]
X_test = [feature(d) for d in test]
y_train = [d for d in data['rating'][:10000]]

In [154]:
mse_bitfwp = []
# Regularized regression
for c in [0.01, 0.1, 1, 10, 100]:
    clf = linear_model.Ridge(c, fit_intercept=False) # MSE + 1.0 l2
    clf.fit(X, y_train)
    theta = clf.coef_
    predictions = clf.predict(X_test)
    mse = sklearn.metrics.mean_squared_error(y_train, predictions)
    mse_bitfwp.append([mse,c , 'bitfwp'])
    print(mse, c)

1.3453519100143996 0.01
1.345351911439946 0.1
1.3453520539686856 1
1.3453662809917355 10
1.3467635116165084 100


# Models Bigrams less tfidf

In [43]:
bigram = defaultdict(int)
punctuation = set(string.punctuation)
for d in data_reviewq:
    r = ''.join([c for c in d.lower() if not c in punctuation])
    words = r.split()
    for j in range(0, len(words)-1):
        bigram[words[j] + ' '+ words[j+1]] +=1
        
counts = [(bigram[w], w) for w in bigram]
counts.sort()
counts.reverse()

words = [x[1] for x in counts[:1000]]

In [48]:
idfwords = []
for i in words:
    #print(i)
    j = idf(i, train)
    #print(j)
    idfwords.append(j)
    #print(idfwords)

In [49]:
### Sentiment analysis

wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

def feature(datum):
    feat = [0]*len(words)
    for i, j in zip(words, idfwords):
        #print(tf(i, datum))
        feat.append(tf(i, datum)* j)
    feat.append(1) #offset
    return feat

X = [feature(d) for d in train]
y_train = [d for d in data['rating'][:10000]]

In [50]:
idfwordsvali = []
for i in words:
    #print(i)
    j = idf(i, validation)
    #print(j)
    idfwordsvali.append(j)

In [51]:
def featurevali(datum):
    feat = [0]*len(words)
    for i, j in zip(words, idfwordsvali):
        #print(tf(i, datum))
        feat.append(tf(i, datum)* j)
    feat.append(1) #offset
    return feat
X_vali = [featurevali(d) for d in validation]

In [ ]:
idfwordstest = []
for i in words:
    #print(i)
    j = idf(i, test)
    #print(j)
    idfwordstest.append(j)

In [ ]:
def featuretest(datum):
    feat = [0]*len(words)
    for i, j in zip(words, idfwordstest):
        #print(tf(i, datum))
        feat.append(tf(i, datum)* j)
    feat.append(1) #offset
    return feat
X_test = [featuretest(d) for d in test]

In [52]:
mse_bitf = []
# Regularized regression
for c in [0.01, 0.1, 1, 10, 100]:
    clf = linear_model.Ridge(c, fit_intercept=False) # MSE + 1.0 l2
    clf.fit(X, y_train)
    theta = clf.coef_
    predictions = clf.predict(X_vali)
    mse = sklearn.metrics.mean_squared_error(y_train, predictions)
    mse_bitf.append([mse,c , 'bitf'])
    print(mse, c)

1.3453519100143996 0.01
1.345351911439946 0.1
1.3453520539686856 1
1.3453662809917355 10
1.3467635116165084 100


# Model unigram with tfidf

In [54]:
import re
uniwordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in data_reviewq:
    r = ''.join([c for c in d.lower() if not c in punctuation])
    for wuni in [t.strip() for t in re.findall(r"[\w']+|[.,!?;]", r)]:
        uniwordCount[wuni] += 1
unicounts = [(uniwordCount[wc], wc) for wc in uniwordCount]
unicounts.sort()
unicounts.reverse()
words = [x[1] for x in unicounts[:1000]]

In [56]:
idfwords = []
for i in words:
    #print(i)
    j = idf(i, train)
    #print(j)
    idfwords.append(j)
    #print(idfwords)

In [57]:
wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

def feature(datum):
    feat = [0]*len(words)
    for i, j in zip(words, idfwords):
        #print(tf(i, datum))
        feat.append(tf(i, datum)* j)
    feat.append(1) #offset
    return feat

X = [feature(d) for d in train]

In [58]:
idfwordsvali = []
for i in words:
    #print(i)
    j = idf(i, validation)
    #print(j)
    idfwordsvali.append(j)

In [59]:
def featurevali(datum):
    feat = [0]*len(words)
    for i, j in zip(words, idfwordsvali):
        #print(tf(i, datum))
        feat.append(tf(i, datum)* j)
    feat.append(1) #offset
    return feat
X_vali = [featurevali(d) for d in validation]

In [ ]:
idfwordstest = []
for i in words:
    #print(i)
    j = idf(i, test)
    #print(j)
    idfwordstest.append(j)

In [ ]:
def featuretest(datum):
    feat = [0]*len(words)
    for i, j in zip(words, idfwordstest):
        #print(tf(i, datum))
        feat.append(tf(i, datum)* j)
    feat.append(1) #offset
    return feat
X_test = [featuretest(d) for d in test]
y_train = [d for d in data['rating'][:10000]]

In [60]:
mse_utfwp = []
# Regularized regression
for c in [0.01, 0.1, 1, 10, 100]:
    clf = linear_model.Ridge(c, fit_intercept=False) # MSE + 1.0 l2
    clf.fit(X, y_train)
    theta = clf.coef_
    predictions = clf.predict(X_vali)
    mse = sklearn.metrics.mean_squared_error(y_train, predictions)
    mse_utfwp.append([mse,c, 'utfwp'])
    print(mse, c)

1.8435331199038523 0.01
1.8418910708574914 0.1
1.8379322157712121 1
1.820328427984602 10
1.7452302239633442 100


# Model unigram less tfidf

In [61]:
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in data_review:
    r = ''.join([c for c in d.lower() if not c in punctuation])
    for w in r.split():
        wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

words = [x[1] for x in counts[:1000]]

In [62]:
idfwords = []
for i in words:
    #print(i)
    j = idf(i, train)
    #print(j)
    idfwords.append(j)
    #print(idfwords)

In [63]:
wordId = dict(zip(words, range(len(words))))
wordSet = set(words)

def feature(datum):
    feat = [0]*len(words)
    for i, j in zip(words, idfwords):
        #print(tf(i, datum))
        feat.append(tf(i, datum)* j)
    feat.append(1) #offset
    return feat

X = [feature(d) for d in train]
y_train = [d for d in data['rating'][:10000]]

In [64]:
idfwordsvali = []
for i in words:
    #print(i)
    j = idf(i, validation)
    #print(j)
    idfwordsvali.append(j)

In [65]:
def featurevali(datum):
    feat = [0]*len(words)
    for i, j in zip(words, idfwordsvali):
        #print(tf(i, datum))
        feat.append(tf(i, datum)* j)
    feat.append(1) #offset
    return feat
X_vali = [featurevali(d) for d in validation]

In [ ]:
idfwordstest = []
for i in words:
    #print(i)
    j = idf(i, test)
    #print(j)
    idfwordstest.append(j)

In [ ]:
def featuretest(datum):
    feat = [0]*len(words)
    for i, j in zip(words, idfwordstest):
        #print(tf(i, datum))
        feat.append(tf(i, datum)* j)
    feat.append(1) #offset
    return feat
X_test = [featuretest(d) for d in test]


In [114]:
mse_utf = []
# Regularized regression
for c in [0.01, 0.1, 1, 10, 100]:
    clf = linear_model.Ridge(c, fit_intercept=False) # MSE + 1.0 l2
    clf.fit(X, y_train)
    theta = clf.coef_
    predictions = clf.predict(X_vali)
    mse = sklearn.metrics.mean_squared_error(y_train, predictions)
    mse_utf.append([mse,c, 'utf'])
    print(mse, c)

1.3453519100143996 0.01
1.345351911439946 0.1
1.3453520539686856 1
1.3453662809917355 10
1.3467635116165084 100


In [ ]:
#+ mse_unibag + mse_bitfwp + mse_bitf mse_utfwp + mse_u

In [141]:
 #mse_bibagwp 

In [142]:
#mse_bibag

In [143]:
#mse_ubagwp

In [144]:
#mse_unibag

In [145]:
#mse_bitfwp

In [146]:
#mse_bitf

In [147]:
 #mse_utfwp

In [148]:
#mse_utf